In [1]:
!pip install yfinance

  Using cached peewee-3.17.6-py3-none-any.whl
  Attempting uninstall: requests
    Found existing installation: requests 2.29.0
    Uninstalling requests-2.29.0:
      Successfully uninstalled requests-2.29.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
conda-repo-cli 1.0.41 requires requests_mock, which is not installed.
spyder 5.4.1 requires pyqt5<5.16, which is not installed.
spyder 5.4.1 requires pyqtwebengine<5.16, which is not installed.
conda-repo-cli 1.0.41 requires clyent==1.2.1, but you have clyent 1.2.2 which is incompatible.
conda-repo-cli 1.0.41 requires nbformat==5.4.0, but you have nbformat 5.9.0 which is incompatible.
conda-repo-cli 1.0.41 requires requests==2.28.1, but you have requests 2.32.3 which is incompatible.


In [47]:
import json
import yfinance as yf

import openai
from dotenv import load_dotenv, find_dotenv

_ = load_dotenv(find_dotenv())

client = openai.Client()


def retorna_cotacao_acao_historica(
        ticker,
        periodo='1mo'
):
    # ex ticker = "ABEV3.SA"
    ticker_obj = yf.Ticker(f'{ticker}.SA')
    # periodo = 1d, 5d, 1mo, 3mo, 6mo, 1y, 2y, 5y, 10y, ytd, max
    hist = ticker_obj.history(period=periodo)['Close']
    hist.index = hist.index.strftime('%Y-%m-%d')
    hist = round(hist, 2)# duas casas depois da virgula
    if len(hist) > 30:
        slice_size = int(len(hist) / 30)
        hist = hist.iloc[::-slice_size][::-1]
    return hist.to_json()


tools = [
    {
        'type': 'function',
        'function': {
            'name': 'retorna_cotacao_acao_historica',                                                   # CHAMA A FUNÇÃO A CIMA
            'description': 'Retorna a cotação diária histórica para uma ação da bovespa',
            'parameters': {
                'type': 'object',
                'properties': {
                    'ticker': {
                        'type': 'string',
                        'description': 'O ticker da ação. Exemplo: "ABEV3" para ambev, "PETR4" para petrobras, etc'
                    },
                    'periodo': {
                        'type': 'string',
                        'description': 'O período que será retornado de dados históriocos \
                                        sendo "1mo" equivalente a um mês de dados, "1d" a \
                                        1 dia e "1y" a 1 ano',
                        'enum': ["1d","5d","1mo","6mo","1y","5y","10y","ytd","max"]
                    }
                }
            }
        }
    }

]

funcoes_disponiveis = {'retorna_cotacao_acao_historica': retorna_cotacao_acao_historica}


mensagens = [{'role': 'user', 'content': 'Qual é a cotação da ambev agora'}]

resposta = client.chat.completions.create(
    messages=mensagens,
    model='gpt-3.5-turbo-0125',
    tools=tools,
    tool_choice='auto'
)


In [48]:
tool_calls = resposta.choices[0].message.tool_calls

if tool_calls:
    mensagens.append(resposta.choices[0].message)
    for tool_call in tool_calls:
        func_name = tool_call.function.name
        function_to_call = funcoes_disponiveis[func_name]
        func_args = json.loads(tool_call.function.arguments)
        func_return = function_to_call(**func_args)
        mensagens.append({
            'tool_call_id': tool_call.id,
            'role': 'tool',
            'name': func_name,
            'content': func_return
        })
    segunda_resposta = client.chat.completions.create(
        messages=mensagens,
        model='gpt-3.5-turbo-0125',
    )
    mensagens.append(segunda_resposta.choices[0].message)


In [52]:
segunda_resposta.choices[0].message.content

'A cotação da ação da Ambev (ABEV3) agora é de R$ 12,54.'

In [45]:
func_name = tool_call.function.name
function_to_call = funcoes_disponiveis[func_name]
func_args = json.loads(tool_call.function.arguments)
func_return = function_to_call(**func_args)


func_return

'{"2024-03-20":12.54}'

In [46]:
func_args

{'ticker': 'ABEV3', 'periodo': '1d'}

In [40]:
tool_call

ChatCompletionMessageToolCall(id='call_VDtRP7mzpaqlvklVZYkQc5jS', function=Function(arguments='{"ticker":"ABEV3","periodo":"1d"}', name='retorna_cotacao_acao_historica'), type='function')